In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import Trainer
from torch import optim

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
import torch

import sys 
import os
sys.path.append(os.path.abspath('../networks'))
sys.path.append(os.path.abspath('../dev'))

from RNN import ModelConfig

preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

2025-06-04 18:33:43.423 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 3126.43it/s]


In [3]:
accuracy = Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

config = ModelConfig(
    input_size=3, # vast
    hidden_size=128,
    num_layers=2,
    output_size=20, # vast
    dropout=0.2,
)

In [4]:
from pathlib import Path

settings = TrainerSettings(
    epochs=10, 
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
    }
)
settings

epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5}

In [5]:
from RNN import RecurrentNeuralNetwork

model = RecurrentNeuralNetwork(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 18:33:57.093 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-183357
2025-06-04 18:33:58.185 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 79.67it/s]
2025-06-04 18:33:59.337 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.7087 test 2.4861 metric ['0.1109']
100%|██████████| 81/81 [00:01<00:00, 79.60it/s]
2025-06-04 18:34:00.441 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.4734 test 2.7625 metric ['0.1062']
2025-06-04 18:34:00.442 | INFO     | mltrainer.trainer:__call__:252 - best loss: 2.4861, current loss 2.7625.Counter 1/5.
100%|██████████| 81/81 [00:01<00:00, 69.39it/s]
2025-06-04 18:34:01.700 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.6898 test 2.5687 metric ['0.1250']
2025-06-04 18:34:01.701 | INFO     | mltrainer.trainer:__call__:252 - best loss: 2.4861, current loss 2.5687.Counte

In [6]:
from RNN import GRUWithAttention

model = GRUWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 18:34:11.003 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-183411
2025-06-04 18:34:11.004 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:03<00:00, 26.94it/s]
2025-06-04 18:34:14.184 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.1419 test 1.4894 metric ['0.5516']
100%|██████████| 81/81 [00:02<00:00, 27.64it/s]
2025-06-04 18:34:17.306 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.9624 test 0.4935 metric ['0.8578']
100%|██████████| 81/81 [00:03<00:00, 26.06it/s]
2025-06-04 18:34:20.597 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3184 test 0.1736 metric ['0.9719']
100%|██████████| 81/81 [00:02<00:00, 27.95it/s]
2025-06-04 18:34:23.690 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.1542 test 0.0870 metric ['0.9844']
100%|██████████| 81/81 [00:02<00:00, 27.40it/s]
2025-06-04 18:34:26.82

In [8]:
from RNN import RecurrentNeuralNetworkWithAttention

model = RecurrentNeuralNetworkWithAttention(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()

2025-06-04 18:36:09.466 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-183609
2025-06-04 18:36:09.467 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 65.91it/s]
2025-06-04 18:36:10.791 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.0716 test 1.4498 metric ['0.5078']
100%|██████████| 81/81 [00:01<00:00, 68.60it/s]
2025-06-04 18:36:12.062 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.2472 test 1.0210 metric ['0.6609']
100%|██████████| 81/81 [00:01<00:00, 66.57it/s]
2025-06-04 18:36:13.365 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.8835 test 0.9264 metric ['0.6484']
100%|██████████| 81/81 [00:01<00:00, 69.51it/s]
2025-06-04 18:36:14.632 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.6887 test 0.7214 metric ['0.7469']
100%|██████████| 81/81 [00:01<00:00, 68.59it/s]
2025-06-04 18:36:15.89